In [ ]:
import numpy as np
import os
import glob
import cv2
import imageio.v2 as imageio
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import skimage
from scipy import io
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier

In [ ]:
ori_A_list = glob.glob('30s_512_grid_PT/A/*.png')
ori_N_list = glob.glob('30s_512_grid_PT/N/*.png')
ori_O_list = glob.glob('30s_512_grid_PT/O/*.png')
ori_I_list = glob.glob('30s_512_grid_PT/~/*.png')

In [ ]:
img_list_ori = []
label_list_ori = [] 
file_list_ori = []
num = 0
for jj, ffname in enumerate(ori_A_list):
    img1 = imageio.imread(ffname)
    
    # resize. (128, 128)로 통일
    img2 = skimage.transform.resize(img1, (224, 224), anti_aliasing=True, preserve_range=False)    
    img2 = img2/np.max(img2) # normalize image to [0 1]
    print(img1.shape, img2.shape)
    
    img_list_ori.append(img2)
    label_list_ori.append(0) # 0 = A, 1 = N, 2 = O, 3 = ~
    file_list_ori.append(ffname)
for jj, ffname in enumerate(ori_N_list):
    img1 = imageio.imread(ffname)
    
    # resize. (128, 128)로 통일
    img2 = skimage.transform.resize(img1, (224, 224), anti_aliasing=True, preserve_range=False)    
    img2 = img2/np.max(img2) # normalize image to [0 1]
    print(img1.shape, img2.shape)
    
    img_list_ori.append(img2)
    label_list_ori.append(1) # 0 = A, 1 = N, 2 = O, 3 = ~
    file_list_ori.append(ffname)
    num = num + 1
    if(num > 5000):
        break
for jj, ffname in enumerate(ori_O_list):
    img1 = imageio.imread(ffname)
    
    # resize. (128, 128)로 통일
    img2 = skimage.transform.resize(img1, (224, 224), anti_aliasing=True, preserve_range=False)    
    img2 = img2/np.max(img2) # normalize image to [0 1]
    print(img1.shape, img2.shape)
    
    img_list_ori.append(img2)
    label_list_ori.append(2) # 0 = A, 1 = N, 2 = O, 3 = ~
    file_list_ori.append(ffname)
for jj, ffname in enumerate(ori_I_list):
    img1 = imageio.imread(ffname)
    
    # resize. (128, 128)로 통일
    img2 = skimage.transform.resize(img1, (224, 224), anti_aliasing=True, preserve_range=False)    
    img2 = img2/np.max(img2) # normalize image to [0 1]
    print(img1.shape, img2.shape)
    
    img_list_ori.append(img2)
    label_list_ori.append(3) # 0 = A, 1 = N, 2 = O, 3 = ~
    file_list_ori.append(ffname)

plt.figure()
plt.subplot(121)
plt.imshow(img1, cmap='jet')
plt.subplot(122)
plt.imshow(img2, cmap='jet')

In [ ]:
from sklearn.model_selection import train_test_split


X_ori = np.stack(img_list_ori, axis=0)
y_ori = np.stack(label_list_ori, axis=0)

X_train_ori, X_test_ori, y_train_ori, y_test_ori = train_test_split(X_ori, y_ori, test_size=0.2, random_state=2334141)

print(X_ori.shape, y_ori.shape)

print(X_train_ori.shape, y_train_ori.shape)
print(X_test_ori.shape, y_test_ori.shape) 

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from keras.applications import ResNet50, DenseNet121, MobileNet
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np  
from tensorflow.keras.models import save_model
from keras.callbacks import ModelCheckpoint  
from tensorflow.keras.models import load_model

In [ ]:
prob_score_list = []
predicted_label_list = []
n_filter = 16
kernel1 = (3, 3)
padding1 = 'same'
strides1 = (1, 1)
input_shape = (128, 128, 3)  # 3개의 채널로 변경
pool_size1 = (2, 2)
Fold = 1
learning_rate = 0.000001
batch_size = 4
n_epochs = 20
num_classes = 4
label_order = ['A', 'N', 'O', '~']
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print(type(X_train_ori), type(y_train_ori))

print(X_train_ori.shape, y_train_ori.shape)
dir_model = 'model_data_epoch'
    
if not os.path.exists(dir_model):
    os.makedirs(dir_model)
    
filepath = dir_model + "/CNN_e{epoch:02d}_valacc{val_accuracy:.2f}.h5"
    #filepath = dir_model + "/CNN_e{epoch:02d}_valf1{val_f1:.2f}.h5"
    
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
    #checkpoint = ModelCheckpoint(filepath, monitor='val_f1', verbose=1, save_best_only=False, mode='max')

callbacks_list = [checkpoint]
# 데이터의 4번째 채널 제거
X_train_ori_rgb = X_train_ori[:, :, :, :3]
X_test_ori_rgb = X_test_ori[:, :, :, :3]
plt.figure()
plt.subplot(121)
plt.imshow(X_train_ori_rgb[0], cmap='jet')
plt.figure()
plt.subplot(121)
plt.imshow(X_test_ori_rgb[0], cmap='jet')
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
history1 = []
for train_index, test_index in kf.split(X_train_ori_rgb, y_train_ori):

    print(type(train_index))
    print(train_index.shape)
    print(test_index)
    model_base = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    model = Sequential([
        model_base,
        Dense(64),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        GlobalAveragePooling2D(keepdims = True, input_shape = 'channels_last'),
        Flatten(),
        Dense(4, activation='softmax')
    ])

    
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    X_train, X_val = X_train_ori_rgb[train_index], X_train_ori_rgb[test_index]

    y_train, y_val = y_train_ori[train_index], y_train_ori[test_index]
    
    history = model.fit(x=X_train, y=y_train, batch_size=16, epochs=20, validation_data=(X_val, y_val), callbacks=callbacks_list)

    # Save each fold model
    model_name = 'model_data/Fold_'+str(Fold)+'.h5'
    model.save(model_name)
    
    plt.figure()
    plt.subplot(121)
    plt.imshow(X_train[0], cmap='jet')
    plt.figure()
    plt.subplot(121)
    plt.imshow(X_val[0], cmap='jet')
    # 테스트 데이터 전처리
    predictions = model.predict(X_test_ori_rgb)
    predicted_labels = np.argmax(predictions, axis=1)
    prob_score_list.append(predictions)
    predicted_label_list.append(predicted_labels)


    cm = confusion_matrix(y_test_ori, predicted_labels)
    print(y_test_ori, predicted_labels)
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_order)
    disp.plot()

    accuracy = accuracy_score(y_test_ori, predicted_labels)
    precision = precision_score(y_test_ori, predicted_labels, average='macro')
    recall = recall_score(y_test_ori, predicted_labels, average='macro')
    f1 = f1_score(y_test_ori, predicted_labels, average='macro')


    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    history1.append(history)
    Fold = Fold + 1



print(f'Mean Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Mean Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Mean Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'Mean f1: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

In [ ]:
# Define base models
def create_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    output =  GlobalAveragePooling2D()(base_model.output)
    output = Dense(4, activation='softmax')(output)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_densenet_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    output =  GlobalAveragePooling2D()(base_model.output)
    output = Dense(4, activation='softmax')(output)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_mobilenet_model():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    output =  GlobalAveragePooling2D()(base_model.output)
    output = Dense(4, activation='softmax')(output)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess data
# Replace X_train_ori_rgb, y_train_ori, X_test_ori_rgb, y_test_ori with your data

# Create ensemble model with voting
ensemble_model = VotingClassifier(estimators=[
    ('resnet', KerasClassifier(build_fn=create_resnet_model, batch_size=16, epochs=20)),
    ('densenet', KerasClassifier(build_fn=create_densenet_model, batch_size=16, epochs=20)),
    ('mobilenet', KerasClassifier(build_fn=create_mobilenet_model, batch_size=16, epochs=20))
], voting='soft')

prob_score_list = []
predicted_label_list = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
label_order = ['A', 'N', 'O', '~']

# Perform 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_count = 1
for train_index, test_index in kf.split(X_train_ori_rgb, y_train_ori):
    print(f"Fold {fold_count}")
    X_train, X_val = X_train_ori_rgb[train_index], X_train_ori_rgb[test_index]
    y_train, y_val = y_train_ori[train_index], y_train_ori[test_index]

    # Train the ensemble model
    ensemble_model.fit(X_train, y_train)
    
    # Validate the ensemble model
    predictions = ensemble_model.predict(X_val)
    print(predictions)

    # Evaluate the ensemble model
    accuracy = accuracy_score(y_val, predictions)
    precision = precision_score(y_val, predictions, average='macro')
    recall = recall_score(y_val, predictions, average='macro')
    f1 = f1_score(y_val, predictions, average='macro')

    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    fold_count += 1

print(f'Mean Accuracy: {np.mean(accuracy_scores):.4f} +/- {np.std(accuracy_scores):.4f}')
print(f'Mean Precision: {np.mean(precision_scores):.4f} +/- {np.std(precision_scores):.4f}')
print(f'Mean Recall: {np.mean(recall_scores):.4f} +/- {np.std(recall_scores):.4f}')
print(f'Mean f1: {np.mean(f1_scores):.4f} +/- {np.std(f1_scores):.4f}')

In [ ]:
def voting_ensemble(model_folder, num_folds):
    models = []
    
    # Load models
    for fold in range(1, num_folds + 1):
        model_path = os.path.join(model_folder, f'Fold_{fold}.h5')
        model = load_model(model_path)
        models.append(model)
    
    # Ensemble predictions
    predictions = np.zeros((len(models), 1196, 4))  # Assuming num_samples and num_classes are known
    
    for i, model in enumerate(models):
        predictions[i] = model.predict(X_test_ori_rgb)  # Assuming x_test is your test data
    
    ensemble_prediction = np.mean(predictions, axis=0)
    
    return ensemble_prediction

# Example usage
model_folder = 'model_data/'
num_folds = 5

voting_result = voting_ensemble(model_folder, num_folds)

In [ ]:
#soft vote
ensemble_vote = []
for i in range (len(voting_result)):
    ensemble_vote.append(np.argmax(voting_result[i]))

In [ ]:
cm = confusion_matrix(y_test_ori, ensemble_vote)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_order)
disp.plot()

accuracy = accuracy_score(y_test_ori, ensemble_vote)
precision = precision_score(y_test_ori, ensemble_vote, average='macro')
recall = recall_score(y_test_ori, ensemble_vote, average='macro')
f1 = f1_score(y_test_ori, ensemble_vote, average='macro')
print(accuracy)
print(precision)
print(recall)
print(f1)
print()